In [1]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install scikit-learn
!pip install sentencepiece
!pip install sacremoses
!pip install evaluate

/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "
/Users/jayjoshi/.zshenv:3: unmatched "


In [2]:
# checkpoint_hindi = "Hate-speech-CNERG/hindi-abusive-MuRIL"
# checkpoint_spanish = "Hate-speech-CNERG/dehatebert-mono-spanish"
# checkpoint_english = "Hate-speech-CNERG/dehatebert-mono-english"
checkpoint_xlm_r = "FineTuned_Model/xlm_r/"



In [3]:
import evaluate

metric_accuracy = evaluate.load("accuracy")

metric_glue = evaluate.load("glue", "mrpc")

metric_precision = evaluate.load("precision")

metric_recall = evaluate.load("recall")

metric_rocauc = evaluate.load("roc_auc")

score_dict = {"accuracy": "none", "glue": "none", "precision": "none", "recall": "none", "roc_auc": "none"}


def compute_metrics(labels):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
    score_dict["accuracy"] = metric_accuracy.compute(predictions=preds, references=labels)
    score_dict["glue"] = metric_glue.compute(predictions=preds, references=labels)
    score_dict["precision"] = metric_precision.compute(predictions=preds, references=labels)
    score_dict["recall"] = metric_recall.compute(predictions=preds, references=labels)
#     score_dict["rocauc"] = metric_rocauc.compute(predictions=preds, references=labels)

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [4]:
import pandas as pd

ds = pd.read_csv("My_Data/en_sentence_templates.txt")

# x = ds["Language"] == "Spanish"
# y = ds["Language"] == "English"
# z = ds["Language"] == "Hindi"

# ds = ds.where(x | y | z)
# condition = ds["Language"] != "English"
# ds = ds.where(condition)

ds.dropna(inplace=True)

ds.rename(columns={"phrase" : "Text", "toxicity": "Label"}, inplace=True)


texts = ds["Text"].tolist()
labels =[0 if i == "nontoxic" else 1 for i in  ds["Label"].tolist()]
# languages = ds["Language"].tolist()

texts[:10]

['hug lesbian',
 'hug gay',
 'hug bisexual',
 'hug transgender',
 'hug trans',
 'hug queer',
 'hug lgbt',
 'hug lgbtq',
 'hug homosexual',
 'hug straight']

In [5]:
texts[1000:1200]

['Being christian is happy',
 'Being christian is good',
 'Being christian is best',
 'Being christian is fantastic',
 'Being christian is wonderful',
 'Being christian is lovely',
 'Being christian is excellent',
 'Being christian is incredible',
 'Being christian is friendly',
 'Being christian is gracious',
 'Being christian is kind',
 'Being christian is caring',
 'Being muslim is great',
 'Being muslim is fun',
 'Being muslim is nice',
 'Being muslim is neat',
 'Being muslim is happy',
 'Being muslim is good',
 'Being muslim is best',
 'Being muslim is fantastic',
 'Being muslim is wonderful',
 'Being muslim is lovely',
 'Being muslim is excellent',
 'Being muslim is incredible',
 'Being muslim is friendly',
 'Being muslim is gracious',
 'Being muslim is kind',
 'Being muslim is caring',
 'Being jewish is great',
 'Being jewish is fun',
 'Being jewish is nice',
 'Being jewish is neat',
 'Being jewish is happy',
 'Being jewish is good',
 'Being jewish is best',
 'Being jewish is fa

In [6]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)


In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# local_files_only=True (Parameter for .from_pretrained to specify taking the tokenizer from local files / .cache)

In [138]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [139]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [140]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

import numpy as np

training_args = TrainingArguments(
    output_dir='./results/cleanData',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs/cleandata/',            # directory for storing logs
    logging_steps=10,
    # use_mps_device=True
)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    
)







PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file config.json from cache at /Users/jayjoshi/.cache/huggingface/hub/models--Hate-speech-CNERG--dehatebert-mono-english/snapshots/25d0e4d9122d2a5c283e07405a325e3dfd4a73b3/config.json
Model config BertConfig {
  "_name_or_path": "Hate-speech-CNERG/dehatebert-mono-english",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "eos_token_ids": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.3,
  "hidden_size": 768,
  "id2label": {
    "0": "NON_HATE",
    "1": "HATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "HATE": 1,
 

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /Users/jayjoshi/.cache/huggingface/hub/models--Hate-speech-CNERG--dehatebert-mono-english/snapshots/25d0e4d9122d2a5c283e07405a325e3dfd4a73b3/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at Hate-speech-CNERG/dehatebert-mono-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [141]:
predictions = trainer.predict(train_dataset)

***** Running Prediction *****
  Num examples = 62016
  Batch size = 64


In [142]:
preds = np.argmax(predictions.predictions, axis=-1)

In [143]:
compute_metrics(predictions.label_ids)

In [144]:
values = list(score_dict.values())
for i in values: print(i)

{'accuracy': 0.6501547987616099}
{'accuracy': 0.6501547987616099, 'f1': 0.531464605018788}
{'precision': 0.8074676816064046}
{'recall': 0.3960794412077124}
none


In [145]:
print("Labels predicted 1: ",(preds==1).sum())
print("Labels predicted 0: ",(preds==0).sum())

Labels predicted 1:  15239
Labels predicted 0:  46777
